In [2]:
import json
db_path = "../dataset"

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")

stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Train

In [3]:
with open(f"{db_path}/train.json", "r") as fp:
    train = json.load(fp)

In [4]:
train.keys()
len(train['premise'])
train['premise']['0'], train['hypothesis']['0'], train['label']['0']

('Pluto rotates once on its axis every 6.39 Earth days;',
 'Earth rotates on its axis once times in one day.',
 'neutral')

In [22]:
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk import pos_tag

class Config:
    stem = False
    stemmer = PorterStemmer()
    lemmatize = False
    lemmatizer = WordNetLemmatizer()
    tokenize = True

    clean_text = True
    regex = r"[^a-zA-Z0-9\-\.\s]"

    pos = True
    pos_tagger = pos_tag

    stopwords = set(stopwords.words('english'))

class Embedder():
    def __init__(self, cfg: Config) -> None:
        self.cfg = cfg

        self.lemmatizer = cfg.lemmatizer
        self.stemmer = cfg.stemmer
        self.stop_words = cfg.stopwords


    def embed(self, sentence: str) -> list[str]:
        if self.cfg.tokenize:
            tokens = nltk.word_tokenize(sentence)

        if self.cfg.lemmatize:
            tokens = [self.lemmatizer.lemmatize(token) for token in tokens]

        if self.cfg.stem:
            tokens = [self.stemmer.stem(token) for token in tokens]

        if self.cfg.clean_text:
            tokens = [re.sub(self.cfg.regex, '', token) for token in tokens]
            tokens = [token for token in tokens if token and token not in self.stop_words]

        if self.cfg.pos:
            tokens = self.cfg.pos_tagger(tokens)
        
        return tokens
    
embedder = Embedder(Config)
train['premise']['0'], embedder.embed(train['premise']['0'])

LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger_eng[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger_eng')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger_eng/[0m

  Searched in:
    - '/root/nltk_data'
    - '/mnt/c/Natural-Language-Processing/venv/nltk_data'
    - '/mnt/c/Natural-Language-Processing/venv/share/nltk_data'
    - '/mnt/c/Natural-Language-Processing/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
